In [1]:
import os

os.environ["CUDA_VISIBLE_DEVICES"] = "0"
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "2"  # 디버그 메시지 끄기

import tensorflow as tf

# gpu 사용 확인
print(tf.test.gpu_device_name())

gpus = tf.config.experimental.list_physical_devices("GPU")
if gpus:
    try:
        # tf.config.experimental.set_visible_devices(gpus[0], "GPU")
        tf.config.experimental.set_memory_growth(gpus[0], True)
    except RuntimeError as e:
        print(e)

from angle_out import out
from tqdm.auto import tqdm
from tensorflow import keras
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.ensemble import RandomForestClassifier
from sklearn import preprocessing
from sklearn.preprocessing import OneHotEncoder, OrdinalEncoder
from keras.optimizers import Adam
from keras.models import Model, Sequential
from keras.layers import GRU, LSTM, Dense, Dropout, Embedding, Input
from keras import layers
import yaml
import xgboost as xgb
import numpy as np
import natsort
import matplotlib.pyplot as plt  # 데이터 시각화
import pickle
import json


def make_random_forest(
    n_estimators=100,
    max_depth=None,
    min_samples_split=1.0,
    random_state=None,
    verbose=1,
):
    model = RandomForestClassifier(
        n_estimators=n_estimators,
        max_depth=max_depth,
        min_samples_split=min_samples_split,
        random_state=random_state,
        verbose=verbose,
    )
    return model


def make_xgboost(verbose=2):
    model = xgb.XGBClassifier(verbosity=verbose)

    return model


def make_lstm():
    model = Sequential()
    model.add(LSTM(128, input_shape=(10, 18), return_sequences=True))
    model.add(Dropout(0.4))
    model.add(LSTM(128, return_sequences=False))
    # model.add(layers.Dense(32, activation="relu"))
    # model.add(layers.TimeDistributed(layers.Dense(13, activation="softmax")))
    model.add(Dense(13, activation="softmax"))

    model.compile(
        optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"]
    )

    print(model.summary())
    return model


def make_gru():
    model = Sequential()
    model.add(
        GRU(
            128,
            input_shape=(10, 18),
            dropout=0.2,
            return_sequences=True,
            name="gru",
        )
    )
    model.add(GRU(128, dropout=0.2, return_sequences=False))
    # model.add(Dense(64, activation="relu"))
    # model.add(Dense(32, activation="relu"))
    model.add(Dense(13, activation="softmax"))

    model.compile(
        optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"]
    )

    return model

/device:GPU:0


/home/pieroot/miniconda3/envs/yolo/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
import yaml

label = []
# 윈도우 사이즈에 맞추어서 저장할 공간
window = 15
# 초당 5개 제한 2초동안 10개

path = "./label"
label_path = "./_mina/data/AI-Hub/poseEstimation/Validation/DOG"

# with open("_mina/project/preLabeled-test02-2023-07-07/config.yaml") as f:
#     conf_yaml = yaml.load(f, Loader=yaml.FullLoader)

#     body_parts = []
#     for part in conf_yaml["bodyparts"]:
#         body_parts.append(part)


# frontRightView = ["RightWrist", "RightArm", "Nose"]
# frontLeftView = ["LeftWrist", "LeftArm", "Nose"]
# backRightView = ["TailTip", "RightFemur", "RightAnkle"]
# backLeftView = ["TailTip", "LeftFemur", "LeftAnkle"]
# frontRightTilt = ["Neck", "RightWrist", "TailStart"]
# frontLeftTilt = ["Neck", "LeftWrist", "TailStart"]
# backRightTilt = ["Neck", "TailStart", "RightAnkle"]
# backLeftTilt = ["Neck", "TailStart", "LeftAnkle"]
# frontRight = ["Neck", "RightArm", "RightWrist"]
# frontLeft = ["Neck", "LeftArm", "LeftWrist"]
# backRight = ["TailStart", "RightFemur", "RightAnkle"]
# backLeft = ["TailStart", "LeftFemur", "LeftAnkle"]
# frontBody = ["Nose", "Neck", "TailStart"]
# backBody = ["Neck", "TailStart", "TailTip"]
# mouth = ["Nose", "MouthCorner", "LowerLip"]
# head = ["Nose", "Forehead", "Neck"]
# tail = ["TailTip", "TailStart", "LeftAnkle"] or ["TailTip", "TailStart", "RightAnkle"]
# direction = ["Nose", "RightArm", "RightFemur"] or ["Nose", "LeftArm", "LeftFemur"]
# label_parts = [
#     frontRightView,
#     frontLeftView,
#     backRightView,
#     backLeftView,
#     frontRightTilt,
#     frontLeftTilt,
#     backRightTilt,
#     backLeftTilt,
#     frontRight,
#     frontLeft,
#     backRight,
#     backLeft,
#     frontBody,
#     backBody,
#     mouth,
#     head,
#     tail,
#     direction,
# ]


body_parts = [
    "Nose",
    "Forehead",
    "MouthCorner",
    "LowerLip",
    "Neck",
    "RightArm",
    "LeftArm",
    "RightWrist",
    "LeftWrist",
    "RightFemur",
    "LeftFemur",
    "RightAnkle",
    "LeftAnkle",
    "TailStart",
    "TailTip",
]

label_parts = [
    ["Nose", "MouthCorner", "LowerLip"],
    ["LowerLip", "Nose", "Forehead"],
    ["Nose", "Forehead", "neck"],
    ["Forehead", "neck", "TailStart"],
    ["neck", "TailStart", "TailTip"],
    ["Forehead", "neck", "RightArm"],
    ["neck", "RightArm", "RightWrist"],
    ["Forehead", "neck", "LeftArm"],
    ["neck", "LeftArm", "LeftWrist"],
    ["neck", "TailStart", "RightFemur"],
    ["neck", "RightFemur", "RightAnkle"],
    ["neck", "TailStart", "LeftFemur"],
    ["neck", "LeftFemur", "LeftAnkle"],
]

import os
import json
import natsort
import numpy as np
from tqdm.auto import tqdm
from angle_out import out

out_dir = f"./label{window}"


label_path = "label"

def make_data(window=10):
    print("make data")
    print(f"label_path > {label_path}")
    out_dir = f"./label{window}_{len(label_parts)}"
    for list_ in tqdm(
        natsort.natsorted(os.listdir(label_path)), desc="label", leave=True, position=0
    ):
        if list_[:5] == "label":
            label_list = f"{label_path}/{list_}/json"
            # print(label_list)
            for l in tqdm(
                natsort.natsorted(os.listdir(label_list)),
                desc=list_[5:],
                leave=True,
                position=1,
            ):
                keypoints = []

                with open(f"{label_list}/{l}") as label_json:
                    label_tmp = json.load(label_json)
                    for annotation in label_tmp["annotations"]:
                        key = annotation["keypoints"]
                        for val in key:
                            # print(val)
                            if key[val] == None:
                                key[val] = {"x": 0, "y": 0}
                        keypoints.append(key)
                tmp = np.array(keypoints)
                meta_data = []
                for data_index in tmp:
                    data_tmp = []
                    for index in data_index.values():
                        data_tmp.append(index)
                    meta_data.append(np.array(data_tmp))

                angle_arr = []
                for m_d in meta_data:
                    angle = out(
                        inputs=m_d, body_parts=body_parts, label_parts=label_parts
                    )
                    angle_arr.append(angle)
                np_tmp = []
                for index in range(window, len(angle_arr)):
                    tmp_wind = angle_arr[index - window : index]

                    np_tmp.append(tmp_wind)
                os.makedirs(f"{out_dir}/{list_[5:]}", exist_ok=True)
                file_name = l.split(".")[0]
                np_save = f"{out_dir}/{list_[5:]}/{file_name}.npy"
                np.save(np_save, np_tmp)

make_data()

label_path > ./_mina/data/AI-Hub/poseEstimation/Validation/DOG


label: 100%|██████████| 26/26 [00:46<00:00,  1.80s/it]


In [ ]:
# iii = 0

# # print("data")
# label_list = []
# for list_ in natsort.natsorted(os.listdir(path)):
#     print(f"{path}/{list_} > ")

#     delta = f"{path}/{list_}"
#     for delta_list in natsort.natsorted(os.listdir(delta)):
#         alpha = f"{delta}/{delta_list}"
#         if alpha.endswith(".csv"):
#             continue
#         elif alpha.endswith(".npy"):
#             # print(f"{alpha} > ")
#             jeta = np.load(alpha, allow_pickle=True)
#             for index in range(jeta.shape[0]):
#                 # print(jeta[index].shape)
#                 if jeta[index].shape != (10, 14):
#                     # print(jeta[index].shape)
#                     # print(alpha)
#                     continue
#                 data.append(jeta[index])
#                 if not list_ in label_list:
#                     label_list.append(list_)
#                 # label.append([label_list.index(list_)])
#                 label.append([list_])

#             pass
#         iii += 1
# # # print(np.shape(data))
# print(f"data shape > {np.shape(data)}")
# x_tmp = np.array(data, dtype=np.float32) / 180.0
# # # x_train = x_train.reshape(-1,4,10)
# # print(f"shape > {y_tmp.shape}")
# print(f"label shape > {np.shape(label)}")
# print(f"label list > {label_list}")
# y_tmp = np.array(label)
# # y_tmp = tf.one_hot(y_tmp, len(label_list))
# encoder = OneHotEncoder(sparse=False)
# # encoder = OrdinalEncoder()
# y_one_hot = encoder.fit_transform(y_tmp)

# pickle.dump(encoder, open("model/encoder.pkl", "wb"))
# with open("model/label.json", "w") as f:
#     json.dump(label_list, f)
# # y_train = tf.keras.utils.to_categorical(y_train, 13)

# x_train, x_test, y_train, y_test = train_test_split(x_tmp, y_one_hot, test_size=0.2, random_state=42)


# # print(f"x shape > {x_tmp.shape}")
# # print(f"y tmp shape > {y_tmp.shape}")
# # print(f"y one hot shape > {y_one_hot.shape}")

# print(f"x train shape > {x_train.shape}")
# print(f"y train shape > {y_train.shape}")
# print(f"x test shape > {x_test.shape}")
# print(f"y test shape > {y_test.shape}")

In [4]:
from keras.models import load_model
import pickle
from tqdm.auto import tqdm
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
import numpy as np
import os
import natsort


window = 10
out_dir = f"./label{window}"
def load_data():
    print("load data")
    x_data = []
    y_data = []

    for label_list in tqdm(
        natsort.natsorted(os.listdir(out_dir)), desc="label", position=0
    ):
        # print(label_list)
        for np_list in tqdm(
            natsort.natsorted(os.listdir(f"{out_dir}/{label_list}")),
            desc=label_list,
            position=1,
        ):
            np_tmp = np.load(f"{out_dir}/{label_list}/{np_list}")
            for tmp in np_tmp:
                x_data.append(tmp)
                y_data.append(label_list)

    x_data = np.array(x_data) / 360.0
    y_data = np.array(y_data)

    # print(f"x_data shape > {x_data.shape}")
    # print(f"y_data shape > {y_data.shape}")

    if not os.path.exists("model/onehot_encoder.pkl"):
        encoder = OneHotEncoder(sparse=False)
        y_one_hot = encoder.fit_transform(y_data.reshape(-1, 1))
        pickle.dump(encoder, open("model/onehot_encoder.pkl", "wb"))
    else:
        encoder = pickle.load(open("model/onehot_encoder.pkl", "rb"))
        y_one_hot = encoder.transform(y_data.reshape(-1, 1))

    x_train, x_test, y_train, y_test = train_test_split(
        x_data, y_one_hot, test_size=0.3, shuffle=True
    )

    return x_train, x_test, y_train, y_test


x_train, x_test, y_train, y_test = load_data()

print(f"x_train shape > {x_train.shape}")
print(f"y_train shape > {y_train.shape}")
print(f"x_test shape > {x_test.shape}")
print(f"y_test shape > {y_test.shape}")


model = load_model(f'model/LSTM_{window}/model.h5')
encoder = pickle.load(open("model/onehot_encoder.pkl", "rb"))

y_test = encoder.inverse_transform(y_test)
# print(x_test[0][0].shape)
for i in range(30):
    test = x_test[i].reshape(1, window, 18)

    y_pred = model.predict(test)
    y_pred = encoder.inverse_transform(y_pred)
    print(y_pred)
    print(y_test[i])


load data


label: 100%|██████████| 13/13 [00:00<00:00, 15.66it/s]


x_train shape > (307794, 10, 18)
y_train shape > (307794, 13)
x_test shape > (131913, 10, 18)
y_test shape > (131913, 13)
1/1 [==============================] - 0s 333ms/step
[['SIT']]
['SIT']
1/1 [==============================] - 0s 12ms/step
[['SIT']]
['SIT']
1/1 [==============================] - 0s 11ms/step
[['BODYLOWER']]
['BODYLOWER']
1/1 [==============================] - 0s 11ms/step
[['FOOTUP']]
['FOOTUP']
1/1 [==============================] - 0s 11ms/step
[['WALKRUN']]
['WALKRUN']
1/1 [==============================] - 0s 11ms/step
[['FOOTUP']]
['FOOTUP']
1/1 [==============================] - 0s 12ms/step
[['BODYSCRATCH']]
['BODYSCRATCH']
1/1 [==============================] - 0s 11ms/step
[['HEADING']]
['HEADING']
1/1 [==============================] - 0s 11ms/step
[['TURN']]
['TURN']
1/1 [==============================] - 0s 11ms/step
[['FOOTUP']]
['BODYLOWER']
1/1 [==============================] - 0s 12ms/step
[['BODYLOWER']]
['BODYLOWER']
1/1 [=======================

In [13]:
x_train, x_test, y_train, y_test = load_data()
print(f"x_train > {x_train.shape}")
print(f"x_train > {x_train[0]}")
print(f"y_test > {y_test.shape}")
print(f"y_test > {y_test[0]}")
onehot_encoder = pickle.load(open("model/onehot_encoder.pkl", "rb"))
print(f"invese > {onehot_encoder.inverse_transform([y_test[0]])}")
# y_test = np.argmax(y_test, axis=1)
# print(f"y_test > {y_test.shape}")
# print(f"y_test > {y_test[0]}")

x_train > (351765, 10, 18)
x_train > [[ 0.28284106  0.11972345 -1.81741783 -1.701639   -1.15378031 -1.14582998
   1.44513505  1.40246465 -0.00681842 -1.74935854 -1.90364462 -1.7631529
   0.10752449  0.21901315  1.14633375 -0.35746489  0.1834515   0.21380812]
 [ 0.27205761  0.12504807 -1.78445548 -1.68201457 -1.14555354 -1.1378508
  -0.53852289  1.41712302 -0.02223897 -1.78672153 -1.90628354 -1.77930482
   0.14056777  0.16734133  1.13389607 -0.32612859  0.24978169  0.25014399]
 [ 0.24126994  0.09013683 -1.75538984 -1.75239074 -1.15033161 -1.14740831
  -0.52617615  1.43272451  1.98448499 -1.79016801 -1.85777468 -1.84096941
   0.16530951  0.20281079  1.15856082 -0.24972779  0.22991372  0.2493306 ]
 [ 0.23256805  0.0530107  -1.78408084 -1.76119614 -1.15239644 -1.15137169
  -0.53509851  1.42270903  1.97957443 -1.75189979 -1.85536817 -1.81483544
   0.19828348  0.26464429  1.16435779 -0.212888    0.15806473  0.26118211]
 [ 0.22938697  0.04727391 -1.81380307 -1.73850879 -1.133038   -1.12773462

In [11]:
import sklearn
from sklearn.model_selection import KFold, cross_val_score
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

import sys


def is_running_in_jupyter():
    """
    Check if the code is running in a Jupyter Notebook environment.
    Returns True if running in Jupyter, False otherwise.
    """
    try:
        if "ipykernel" in sys.modules:
            # Running in IPython (Jupyter) environment
            return True
        elif "get_ipython" in globals():
            # Running in IPython (Jupyter) environment
            return True
        else:
            # Not running in IPython (Jupyter) environment
            return False
    except Exception:
        return False


# # Check if running in Jupyter Notebook
# if is_running_in_jupyter():
#     print("Running in Jupyter Notebook.")
# else:
#     print("Running in a regular Python script.")


def score_check():
    model = make_lstm()
    x_train, x_test, y_train, y_test = load_data()
    model_name = "LSTM"
    os.makedirs(f"model/{model_name}", exist_ok=True)

    verbose = 0
    if is_running_in_jupyter():
        verbose = 2

    early_stopping = EarlyStopping(
        monitor="val_loss",
        patience=10,
        verbose=verbose,
        mode="auto",
        restore_best_weights=True,
    )
    model_checkpoint = ModelCheckpoint(
        filepath=f"model/{model_name}/best_model.h5",
        monitor="val_loss",
        verbose=verbose,
        save_best_only=True,
    )
    collback = [early_stopping, model_checkpoint]
    kf = KFold(n_splits=5, shuffle=True, random_state=42)
    try:
        scores = []
        for train_index, test_index in kf.split(x_train):
            model = make_lstm()
            X_train, X_test = x_train[train_index], x_train[test_index]
            Y_train, Y_test = y_train[train_index], y_train[test_index]
            model.fit(
                X_train,
                Y_train,
                validation_data=(X_test, Y_test),
                epochs=200,
                batch_size=32,
                verbose=1,
                callbacks=collback,
            )
            loss, acc = model.evaluate(x_test, y_test)
            print(f"loss > {loss}")
            print(f"acc > {acc}")
            scores.append(acc)
            model.save(f"model/{model_name}/model_{len(scores)}.h5")

        plt.figure(figsize=(10, 10))
        plt.boxplot(scores)
        plt.xlabel("fold")
        plt.ylabel("accuracy")
        # plt.plot(history.history["loss"])
        # plt.plot(history.history["accuracy"])
        plt.show()

    except KeyboardInterrupt:
        print("KeyboardInterrupt")
        model = keras.models.load_model(f"model/{model_name}/best_model.h5")
    finally:
        model.save(f"model/{model_name}/model.h5")

    # score = model.score(x_train, x_train)

    # model.evaluate(x_tmp, y_tmp)
    # print(f"score > {score}")
    # print(f"model accuracy score > {accuracy_score(y_test, model.predict(X_test))}")

    # import time

    # start = time.time()
    # print(f"x_test[0] > {X_test[0]}")
    # y_pred = model.predict([X_test[0]])
    # model.
    # run_time = time.time() - start
    # print(f"predict time > {run_time}")

    # params = model.get_params()

    # params["score"] = score
    # params["pred_time"] = run_time
    # json_data = json.dumps(params)
    # with open(f"./model/depth_{max_depth}_{n_estimators}.json", "w") as json_file:
    # json.dump(params, json_file)


# print(f"실패한 갯수 > {faild/len(x_train)}")
# print(f"{x_train[0]}")


# y_pred = model.predict(x_tmp)
# print(f"예상치 > {y_pred}")
# print(f"실제값 > {y_tmp[0]}")

score_check()

Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_9 (LSTM)               (None, 10, 128)           75264     
                                                                 
 dropout_5 (Dropout)         (None, 10, 128)           0         
                                                                 
 lstm_10 (LSTM)              (None, 128)               131584    
                                                                 
 dense_4 (Dense)             (None, 13)                1677      
                                                                 
Total params: 208,525
Trainable params: 208,525
Non-trainable params: 0
_________________________________________________________________
None


label: 100%|██████████| 13/13 [00:06<00:00,  2.03it/s]


x_data shape > (439707, 10, 18)
y_data shape > (439707,)
Model: "sequential_6"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_11 (LSTM)              (None, 10, 128)           75264     
                                                                 
 dropout_6 (Dropout)         (None, 10, 128)           0         
                                                                 
 lstm_12 (LSTM)              (None, 128)               131584    
                                                                 
 dense_5 (Dense)             (None, 13)                1677      
                                                                 
Total params: 208,525
Trainable params: 208,525
Non-trainable params: 0
_________________________________________________________________
None
Epoch 1/200
8791/8795 [============================>.] - ETA: 0s - loss: 1.7727 - accuracy: 0.4250
Epoch 1: val_loss i